In [93]:
import lottomain as lm
import importlib as i
i.reload(lm)

# NN
from keras.layers import LSTM, Dense, Bidirectional, Dropout
# optimizer
from tensorflow.keras.optimizers import Adam

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# normalize
from tensorflow.keras.constraints import min_max_norm

import tensorflow as tf

In [58]:
df = lm.getLotto(link='https://bet.szerencsejatek.hu/cmsfiles/otos.xls',
                mode="otos")

In [80]:
# filter last 30 yeras
ÉVEK = 30
df = df[0:(ÉVEK*52)]

# get the x last week
WEEK = 52*4
#dfNumpy = df[:WEEK][["1","2","3","4","5"].values
dfNumpy = df[:WEEK][["1","2","3","4","5"]]

In [81]:
df

,1,2,3,4,5
Év,,,,,
2023,18,42,44,48,86
2023,25,36,38,51,89
2023,12,37,50,74,77
2023,24,27,50,54,71
2023,29,35,36,48,83
...,...,...,...,...,...
1993,31,44,52,54,73
1993,1,4,39,54,79
1993,6,22,65,78,79


In [60]:
#scaler?
#### It has to be done with the original df 
# between -1 and 1
scaler, BATCH, EPOCS = lm.StandardScaler().fit(df.values), 300, 300
# between 0 and 1
scaler, BATCH, EPOCS = lm.MinMaxScaler().fit(df.values), 500, 100

# here I just scaling the last 3 years
transformed_dataset = scaler.transform(dfNumpy.values)
transformed_df = lm.pd.DataFrame(data=transformed_dataset, index=dfNumpy.index)

In [61]:
#trainY,trainX = dfNumpy[0:1],dfNumpy[1:WEEK]
trainY,trainX = transformed_df[0:1].values,transformed_df[1:WEEK].values

In [62]:
lm.minMaxValuePrint(transformed_dataset)

 minimum value 0.0, maximum value 0.8064516129032258 column 1
 minimum value 0.012987012987012984, maximum value 0.8831168831168832 column 2
 minimum value 0.02631578947368421, maximum value 0.9473684210526315 column 3
 minimum value 0.013513513513513514, maximum value 0.9864864864864865 column 4
 minimum value 0.03278688524590162, maximum value 1.0 column 5


IndexError: index 5 is out of bounds for axis 1 with size 5

In [84]:
# init 
# Amount of games we need to take into consideration for prediction
window_length = 9
window_length
number_of_features = 5 # how many balls
number_of_rows = 4*52 # based on last 3 years   

X = lm.np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = lm.np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [90]:
# Set output constraints
min_value = 0.0
max_value = 1.0

model = Sequential()
parameters = 240
model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))
# reduce overfitting
model.add(Dropout(0.2))
# second layer
model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))
# second dropout
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))

model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = False)))

# Define the output constraints as a custom constraint
output_constraint = min_max_norm(min_value=min_value, max_value=max_value)

# add Dense layer
model.add(Dense(59, activation='relu'))

# output
model.add(Dense(number_of_features, activation='sigmoid'))

# Apply the constraint to the Dense layer
model.layers[-1].kernel_constraint = output_constraint
model.layers[-1].bias_constraint = output_constraint

model.layers[-2].kernel_constraint = output_constraint
model.layers[-2].bias_constraint = output_constraint

# complier
# model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])
# New model complier

In [109]:
# New model complier
def sequence_accuracy(y_true, y_pred):
    # Convert the predicted and true sequences to tensors
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)

    # Compute element-wise matches
    matches = tf.cast(tf.equal(y_true, y_pred), tf.float32)
    #print(matches)

    # Calculate the accuracy for each element in the sequence
    per_element_accuracy = tf.reduce_mean(matches, axis=-1)
    #print(per_element_accuracy)

    # Calculate the overall accuracy
    accuracy = tf.reduce_mean(per_element_accuracy)
    #print(accuracy)

    return accuracy

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=["accuracy", sequence_accuracy])


In [105]:
y_t = [1,2,3]
y_p = [1,2,4]
a = sequence_accuracy(y_t, y_p)

tf.Tensor([1. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor(0.6666667, shape=(), dtype=float32)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'value'

In [110]:
EPOCS = 100
# fit it
history = model.fit(x=X, y=y, batch_size=BATCH, epochs=EPOCS, verbose=2)

Epoch 1/100
Tensor("Cast_3:0", shape=(None, 5), dtype=float32)
Tensor("Mean:0", shape=(None,), dtype=float32)
Tensor("Mean_1:0", shape=(), dtype=float32)
Tensor("Cast_3:0", shape=(None, 5), dtype=float32)
Tensor("Mean:0", shape=(None,), dtype=float32)
Tensor("Mean_1:0", shape=(), dtype=float32)
1/1 - 13s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 13s/epoch - 13s/step
Epoch 2/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 822ms/epoch - 822ms/step
Epoch 3/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 825ms/epoch - 825ms/step
Epoch 4/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 828ms/epoch - 828ms/step
Epoch 5/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 823ms/epoch - 823ms/step
Epoch 6/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 818ms/epoch - 818ms/step
Epoch 7/100
1/1 - 1s - loss: 0.0425

Epoch 71/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 928ms/epoch - 928ms/step
Epoch 72/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 921ms/epoch - 921ms/step
Epoch 73/100
1/1 - 1s - loss: 0.0424 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 902ms/epoch - 902ms/step
Epoch 74/100
1/1 - 1s - loss: 0.0424 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 913ms/epoch - 913ms/step
Epoch 75/100
1/1 - 1s - loss: 0.0425 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 942ms/epoch - 942ms/step
Epoch 76/100
1/1 - 1s - loss: 0.0424 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 916ms/epoch - 916ms/step
Epoch 77/100
1/1 - 1s - loss: 0.0424 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 908ms/epoch - 908ms/step
Epoch 78/100
1/1 - 1s - loss: 0.0424 - accuracy: 0.7789 - sequence_accuracy: 0.0000e+00 - 892ms/epoch - 892ms/step
Epoch 79/100
1/1 - 1s - loss: 0.0423 - accuracy: 0.7789 - sequence_accuracy: 0.0

KeyboardInterrupt: 

In [89]:
# the model is done now. Lets take the last X games and try to predict
BASEDON = 7
#to_pred = df[1:BASEDON][["1","2","3","4","5","6"]].values
# predict based on the last x weeks
to_pred = df[0:BASEDON][["1","2","3","4","5"]].values
# scaling
to_pred = scaler.transform(to_pred)
pred = df[0:1][["1","2","3","4","5"]].values

y_pred = model.predict(lm.np.array([to_pred]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 28ms/step
The predicted numbers in the last lottery game are: [18 32 44 55 69]


In [74]:
df

,1,2,3,4,5
Év,,,,,
2023,18,42,44,48,86
2023,25,36,38,51,89
2023,12,37,50,74,77
2023,24,27,50,54,71
2023,29,35,36,48,83
...,...,...,...,...,...
1993,31,44,52,54,73
1993,1,4,39,54,79
1993,6,22,65,78,79


In [54]:
chance = lm.minMaxValuePrint(df.values, columns=5)

 minimum value 1, maximum value 63 column 1
 minimum value 2, maximum value 79 column 2
 minimum value 8, maximum value 84 column 3
 minimum value 15, maximum value 89 column 4
 minimum value 29, maximum value 90 column 5


In [56]:
def find_largest_overlap(ranges):
    largest_overlap = set()
    for i in range(len(ranges)):
        for j in range(i+1, len(ranges)):
            overlap = set(range(ranges[i][0], ranges[i][1]+1)) & set(range(ranges[j][0], ranges[j][1]+1))
            if len(overlap) > len(largest_overlap):
                largest_overlap = overlap

    return largest_overlap

# Számsorok
number_ranges = [
    (1, 63),
    (2, 79),
    (8, 84),
    (15, 89),
    (29, 90)
]

# Legnagyobb átfedés meghatározása
overlap = find_largest_overlap(number_ranges)

print("Legnagyobb átfedés:")
print(overlap)

Legnagyobb átfedés:
{8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79}


In [55]:
chance

[63, 78, 77, 75, 62]

In [37]:
t = 1/62 * 1/63 * 1/75 * 1/77 * 1/78
t = 1/62 * 1/63 * 1/75 * 1/77 * 1/78

In [50]:
1/t

1759457700.0

In [38]:
print('{:.16f}'.format(t))

0.0000000005683569


In [49]:
5!

SyntaxError: invalid syntax (992583946.py, line 1)

In [53]:
def calculate_probabilities(ranges):
    # Számsorok
    numbers = []
    for r in ranges:
        numbers.append(list(range(r[0], r[1]+1)))

    # Lehetséges kombinációk száma
    total_combinations = 1
    for nums in numbers:
        total_combinations *= len(nums)

    # Átfedés kezelése
    for i in range(len(ranges)):
        overlap = set(numbers[i]) & set(numbers[i-1])
        if overlap:
            total_combinations -= 1

    # Esély kiszámítása
    probability = 1 / total_combinations

    return probability

# Számsorok
number_ranges = [
    (1, 63),
    (2, 79),
    (8, 84),
    (15, 89),
    (29, 90)
]

# Esély kiszámítása
result = calculate_probabilities(number_ranges)
print('{:.30f}'.format(result))

0.000000000568356944780078967022
